In [ ]:
import scrapy
from scrapy.selector import Selector

class ZakvaskiSpider(scrapy.Spider):
    name = 'ZakvaskiSpider'
    start_urls = ['https://pro-syr.ru/zakvaski-dlya-syra/mezofilnye/']

    def parse(self, response):
        links = [data.get() for data in response.css('div.nameproduct a::attr(href)')]
        for link in links:
            yield response.follow(link, callback=self.parse_page)

        pagination_list = [pagination.get() for pagination in response.css('ul.pagination li')]

        for item in pagination_list:
            item = Selector(text=item)
            item_text = item.css('a::text').get()
            if item_text == '>':
                link = item.css('a::attr(href)').get()
                yield response.follow(link, callback=self.parse)

    def parse_page(self, response):
        name = response.css('div.col-sm-6 h1::text').get()
        price = response.css('span.autocalc-product-price::text').get()
        availability = response.css('b.outstock::text').get()

        yield {
            'name': name,
            'price': price,
            'availability': availability,
        }